In [ ]:
import datacube
from datacube.storage.masking import mask_invalid_data
from datacube.api.query import Query
dc = datacube.Datacube()

In [ ]:
# Get user input

# Select a product
p = 'LC08_scene'

# Select band to red, green, or blue
r = 6
g = 5
b = 4

# Select available regions
nwt = False            # example of burn 
vi = True             # example of coastal area
ll = False             # example of surface mine 



# Select time start and end
t1 = '2018-05-05'
t2 = '2018-05-06'

In [ ]:
# Prepare the DB query

r, g, b = map(lambda x: x + 1,[r, g, b])

if nwt :
    lat = (62.3768, 63.3062)
    lon = (-114.7038, -112.3989)
elif vi :
    lat = (47.9432, 49.4273)
    lon = (-123.7845, -121.2920)
elif ll :
    lat = (49.7178, 50.8349)
    lon = (-122.2683, -119.1079)
# Petawawa is default selection
else :
    lat = (45.4499, 46.6034)
    lon = (-78.4944, -76.7130)    

    
LC08_Bands = ['coastal_aerosol', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'panchromatic', 'cirrus', 'lwir1', 'lwir2', 'quality']


query = { 
    'product' : p
    , 'latitude' : lat
    , 'longitude' : lon
    , 'measurements' : [ LC08_Bands[r], LC08_Bands[g], LC08_Bands[b] ]
    , 'resolution' : (-0.0025, 0.0025)
}

if (t1 != '') and (t2 != '') :
    query['time'] = (t1, t2)

In [ ]:
# Explore the datacube for datasets of interest
dc.find_datasets(**query)

In [ ]:
# Query the datacube and load dataset to memory
data = dc.load( **query)

In [ ]:
fake_saturation = 400000

# Sets all `nodata` values to ``nan``.
data = mask_invalid_data(data)

# Isolate the color dimension in an xarray.DataArray, use transpose to make color the last dimension
rgb = (data.to_array(dim='color'))
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1])) 

# Filter out pixels where any band is 'saturated'
rgb = rgb.where((rgb <= fake_saturation).all(dim='color'))

# Scale to [0, 1] range for imshow
rgb /= fake_saturation  

In [ ]:
rgb.plot.imshow( x = data.crs.dimensions[1]
                , y = data.crs.dimensions[0]
                , col = 'time'
                , col_wrap = 1
                , size = 10
                , add_colorbar = False
               )